In [49]:
from __future__ import print_function
import os
import numpy as np
import numpy.ma as ma
import Ngl, Nio
from datetime import date, timedelta

In [50]:
from netCDF4 import Dataset
nc_obj = Dataset('./wind.nc')

In [51]:
print(nc_obj)
print(nc_obj.variables.keys())
for i in nc_obj.variables.keys():
    print(i)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    Conventions: CF-1.6
    history: 2020-01-17 05:52:24 GMT by grib_to_netcdf-2.15.0: /opt/ecmwf/eccodes/bin/grib_to_netcdf -o /cache/data3/adaptor.mars.internal-1579240230.411107-12784-7-6f0a8c7f-ccba-4a4f-a279-6e4653b116d1.nc /cache/tmp/6f0a8c7f-ccba-4a4f-a279-6e4653b116d1-adaptor.mars.internal-1579240230.4115527-12784-4-tmp.grib
    dimensions(sizes): longitude(720), latitude(361), number(10), time(132)
    variables(dimensions): float32 longitude(longitude), float32 latitude(latitude), int32 number(number), int32 time(time), int16 u10_0001(time,number,latitude,longitude), int16 v10_0001(time,number,latitude,longitude), int16 u10_0005(time,number,latitude,longitude), int16 v10_0005(time,number,latitude,longitude)
    groups: 
dict_keys(['longitude', 'latitude', 'number', 'time', 'u10_0001', 'v10_0001', 'u10_0005', 'v10_0005'])
longitude
latitude
number
time
u10_0001
v10_0001
u10_00

In [52]:
u10_0001 = nc_obj.variables['u10_0001'][:]
v10_0001 = nc_obj.variables['v10_0001'][:]
longitude = nc_obj.variables['longitude'][:]
latitude = nc_obj.variables['latitude'][:]
times = nc_obj.variables['time'][:]

## Draw a picutre

In [53]:
index = 0

In [54]:
u10_to_draw = np.mean(u10_0001[index, :, :, :], axis=0)
v10_to_draw = np.mean(v10_0001[index, :, :, :], axis=0)
time_to_draw = times[index]

In [55]:
def draw_wind_by_month(time, lat, lon, u, v):
    # time is int32
    
    # calculate file name
    days = time/24 
    start = date(1900,1,1)
    delta = timedelta(days)
    offset = start + delta
    months = offset.isoformat()[:7]
    path = "./wind/wind10_" + months
    title_name = months
    
    wind_speed = np.sqrt(u**2 + v**2)
    
    wks_type = "svg"
    wks = Ngl.open_wks(wks_type, path)

    cnres                             = Ngl.Resources()
    cnres.nglDraw                     = False
    cnres.nglFrame                    = False

#     cmap = Ngl.read_colormap_file("WhiteBlueGreenYellowRed")

    cnres.cnLinesOn                   = False
    cnres.cnLineLabelsOn              = False
    cnres.cnFillOn                    = True
#     cnres.cnFillPalette               = cmap[15:,:]
    cnres.cnFillPalette   = "WhiteBlueGreenYellowRed"
    cnres.cnLevelSelectionMode = "ManualLevels"
    cnres.cnMinLevelValF         = 3
    cnres.cnMaxLevelValF         = 16
    cnres.cnLevelSpacingF        = 1
     
    cnres.lbOrientation               = "horizontal"

    cnres.mpFillOn                    = False
    cnres.mpGeophysicalLineColor      = "Grey18"
    cnres.mpGeophysicalLineThicknessF = 1.5

    cnres.sfXArray                    = lon
    cnres.sfYArray                    = lat

    vcres                         = Ngl.Resources()
    vcres.nglDraw                 = False
    vcres.nglFrame                = False

    vcres.vfXArray                = lon
    vcres.vfYArray                = lat

    vcres.vcFillArrowsOn          = True
    vcres.vcRefMagnitudeF         = 30.0             # define vector ref mag
    vcres.vcRefLengthF            = 0.02             # define length of vec ref
    vcres.vcMinFracLengthF        = 0.3
    vcres.vcMinDistanceF          = 0.02
    vcres.vcRefAnnoOrthogonalPosF = -0.20
    vcres.vcRefAnnoFontHeightF    = 0.005

    cnres.tiMainString    = title_name

    #---Draw fully opaque vectors
    uv_plot  = Ngl.vector(wks, u, v, vcres)
    spd_plot = Ngl.contour_map(wks, wind_speed, cnres)
    Ngl.overlay(spd_plot,uv_plot)
    Ngl.maximize_plot(wks, spd_plot)
    Ngl.draw(spd_plot)
    Ngl.frame(wks)

    #---This time make vectors partially transparent
#     vcres.vcGlyphOpacityF = 0.3
#     cnres.tiMainString    = title_name

#     uv_plot  = Ngl.vector(wks, u, v,  vcres)
#     spd_plot = Ngl.contour_map(wks, wind_speed, cnres)

#     Ngl.overlay(spd_plot,uv_plot)
#     Ngl.maximize_plot(wks, spd_plot)
#     Ngl.draw(spd_plot)
#     Ngl.frame(wks)

    Ngl.delete_wks(wks)

In [56]:
# draw_wind_by_month(time_to_draw, latitude, longitude, u10_to_draw, v10_to_draw)

## Draw all months

In [57]:
index = 0
for time in times:
    time_to_draw = time
    u10_to_draw = np.mean(u10_0001[index, :, :, :], axis=0)
    v10_to_draw = np.mean(v10_0001[index, :, :, :], axis=0)
    draw_wind_by_month(time_to_draw, latitude, longitude, u10_to_draw, v10_to_draw)
    index = index +1